In [38]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
import pandas as pd
import numpy as np
import copy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import clean as clean

In [40]:
# Load the raw data
dt_raw = pd.read_csv('../data/TBI PUD 10-08-2013.csv')
dt_clean = clean.clean_data('../data/TBI PUD 10-08-2013.csv', remove_feats_after_ct=True, remove_TBI_rows_with_nan=True,threshold=0.5,rm_feats=True,remove_GCS_total_mismatch=True)

In [41]:
print(dt_clean.columns)

Index(['PatNum', 'EmplType', 'Certification', 'InjuryMech',
       'High_impact_InjSev', 'Amnesia_verb', 'LOCSeparate', 'Seiz', 'ActNorm',
       'HA_verb', 'Vomit', 'Dizzy', 'Intubated', 'Paralyzed', 'Sedated',
       'GCSEye', 'GCSVerbal', 'GCSMotor', 'GCSTotal', 'GCSGroup', 'AMS',
       'SFxPalp', 'FontBulg', 'SFxBas', 'Hema', 'Clav', 'ClavFace', 'ClavNeck',
       'ClavFro', 'ClavOcc', 'ClavPar', 'ClavTem', 'NeuroD', 'OSI', 'Drugs',
       'AgeInMonth', 'AgeinYears', 'AgeTwoPlus', 'Gender', 'Ethnicity', 'Race',
       'Observed', 'EDDisposition', 'PosIntFinal'],
      dtype='object')


In [42]:
# Split into training and test sets
train_df, test_df = train_test_split(dt_clean, test_size=0.2, random_state=42)

# Separate features (X) and target (y)
y_train = train_df["PosIntFinal"]
X_train = train_df.drop(columns=["PosIntFinal"])
y_test = test_df["PosIntFinal"]
X_test = test_df.drop(columns=["PosIntFinal"])

# Train a Random Forest model with class weighting to address imbalance
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight="balanced")
rf_model.fit(X_train, y_train)

# Predictions
y_pred = rf_model.predict(X_test)
y_proba = rf_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# save the model in results folder
np.save('../results/randomforest.npy', y_proba)

In [43]:
# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nROC AUC Score:")
print(roc_auc_score(y_test, y_proba))

# Feature Importance
feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nTop 10 Feature Importances:")
print(feature_importances.head(10))

Confusion Matrix:
[[8244   10]
 [ 106   48]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      8254
         1.0       0.83      0.31      0.45       154

    accuracy                           0.99      8408
   macro avg       0.91      0.66      0.72      8408
weighted avg       0.98      0.99      0.98      8408


ROC AUC Score:
0.9739771979897981

Top 10 Feature Importances:
          Feature  Importance
42  EDDisposition    0.344443
20            AMS    0.077687
8         ActNorm    0.062070
18       GCSTotal    0.060269
16      GCSVerbal    0.057429
6     LOCSeparate    0.052580
19       GCSGroup    0.024723
15         GCSEye    0.024575
35     AgeInMonth    0.023595
0          PatNum    0.023489
